In [64]:
import pandas as pd
import os

In [65]:
#Select and delete all of the appended TOWN (UPDATE PER ITERATION) data from the MAG dataset

arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="TransportationMasterPlans_2024",
    selection_type="ADD_TO_SELECTION",
    where_clause="PlanSource IS NULL OR PlanSource = ''",
    invert_where_clause=None
)

arcpy.management.DeleteRows(
    in_rows="TransportationMasterPlans_2024"
)


<Result 'TransportationMasterPlans_2024'>

In [66]:
#Set datasource and datasets
#All variables needed for each iteration are located here

#Set workspace
arcpy.env.workspace = "C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb"
coordinate_system = 26912

#Add a section to download data from link and add it to the .gdb with a new name?
#datasource = r'https://services7.arcgis.com/8iAE9RYcVCN6ZjpB/arcgis/rest/services/FutureFuncClass/FeatureServer/0'

#This will add the dataset, once added to the map, to the geodatabase for the project (RESET the name) 
#with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_12N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-111.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'):
 #   arcpy.conversion.FeatureClassToGeodatabase(
  #      Input_Features="EagleMountain_Future_Functional_Class",
   #     Output_Geodatabase=r"C:\Users\eclinton\Documents\ArcGIS\Projects\MAG_TransportationUpdate\MAG_TransportationUpdate.gdb"
    #)
    
#transit_data = "C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\EagleMountain_Future_Functional_Class"
municipalities = "Municipalities_Working"
transit_masterplan = "Arterial_Collector_Rds"

county = "C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\Utah_Co"

#When applied, this allows you to re-run the code and delete its outputs when you re-run
def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

#Delete the clipped output (RESET with new town name variable if you run multiple times on each dataset)
deleteIfExists('C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\County_NoTowns')
deleteIfExists('C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\County_Clipped')
deleteIfExists('C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\Utah_Co_Outliers')
deleteIfExists('C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\Utah_Co_EraseTowns')
deleteIfExists('C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\TransitOutliers_Appended')
deleteIfExists('C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\TransitOutlier_Clipped')
deleteIfExists('C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\Transit_Outliers')
#deleteIfExists(transit_clipped)
#deleteIfExists(transit_toAppend)

In [67]:
#Find which features lie outside of the current municipal boundaries for future use (RESET for next case) 
arcpy.analysis.PairwiseErase(
    in_features="Utah_Co",
    erase_features=municipalities,
    out_feature_class=r"C:\Users\eclinton\Documents\ArcGIS\Projects\MAG_TransportationUpdate\MAG_TransportationUpdate.gdb\County_NoTowns",
    cluster_tolerance=None
)


<Result 'C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\County_NoTowns'>

In [68]:
#Erase the annexation areas from the county layer that has had the towns erased
arcpy.analysis.PairwiseErase(
    in_features="County_NoTowns",
    erase_features="MAG_AnnexAreas",
    out_feature_class=r"C:\Users\eclinton\Documents\ArcGIS\Projects\MAG_TransportationUpdate\MAG_TransportationUpdate.gdb\Utah_Co_Outliers",
    cluster_tolerance=None
)

<Result 'C:\\Users\\eclinton\\Documents\\ArcGIS\\Projects\\MAG_TransportationUpdate\\MAG_TransportationUpdate.gdb\\Utah_Co_Outliers'>

In [69]:
#Clip the transit data to the erased county boundaries
with arcpy.EnvManager(outputCoordinateSystem=coordinate_system):
    arcpy.analysis.Clip("Arterial_Collector_Rds", "Utah_Co_Outliers", 'TransitOutlier_Clipped')

transit_clipped = 'TransitOutlier_Clipped'

In [70]:
#Examine the transit dataframe
fields = [f.name for f in arcpy.ListFields(transit_clipped)]
print(fields)

['OBJECTID_1', 'Shape', 'OBJECTID', 'NAME', 'TYPE', 'STATUS', 'SETBACK', 'SOURCE', 'GLOBALID', 'LASTUPDATE', 'LASTEDITOR', 'CREATED_US', 'CREATED_DA', 'LAST_EDITE', 'LAST_EDI_1', 'Shape_Length']


In [71]:
# Set local variables and fields to be dropped from the previous step's output (RESET with new town name) 
transit_toAppend = "TransitOutliers"

#RESET drop fields based on previous output
inFeatures = transit_clipped
outFeatureClass = transit_toAppend
dropFields =['OBJECTID', 'STATUS', 'SETBACK', 'SOURCE', 'GLOBALID', 'LASTUPDATE', 'LASTEDITOR', 'CREATED_US', 'CREATED_DA', 'LAST_EDITE', 'LAST_EDI_1',]

# Execute CopyFeatures to make a new copy of the feature class
#  Use CopyRows if you have a table
arcpy.CopyFeatures_management(inFeatures, outFeatureClass)
 
# Execute DeleteField
arcpy.DeleteField_management(outFeatureClass, dropFields)

print("You should now have a layer with only relevant or required fields from the original dataset")

You should now have a layer with only relevant or required fields from the original dataset


In [72]:
#Change any field names to match what is in the MAG master data

arcpy.management.AlterFields(
    in_table=transit_toAppend,
    field_description="NAME StreetName 'Street Name' TEXT 255 # #;TYPE CityRdType 'City Road Type' TEXT 255 # #"
)

<Result 'TransitOutliers'>

In [73]:
#Add any missing fields to match the MAG master data

arcpy.management.AddFields(
    in_table=transit_toAppend,
    field_description="'LanesToday' SHORT 'Lanes Today' # # #;'LanesPlan' SHORT 'Lanes Plan' # # #;'WidthToday' SHORT 'Width Today' # # #;'WidthPlan' SHORT 'Width Plan' # # #;City TEXT City # # #;County TEXT County # # #;'GenRdType' TEXT 'Generalized Road Type' # # #;'PlanYear' LONG 'Plan Year' # # #;'PlanSource' TEXT 'Plan Source' # # #;'DataSource' TEXT 'Data Source' # # #;'AnnexArea' TEXT 'Future Annexation' # # #",
    template=None
)

#field_description="'LanesToday' SHORT 'Lanes Today' # # #;'LanesPlan' SHORT 'Lanes Plan' # # #;'WidthToday' SHORT 'Width Today' # # #;'WidthPlan' SHORT 'Width Plan' # # #;City TEXT City # # #;County TEXT County # # #;'GenRdType' TEXT 'Generalized Road Type' # # #;'PlanYear' LONG 'Plan Year' # # #;'PlanSource' TEXT 'Plan Source' # # #;'DataSource' TEXT 'Data Source' # # #;'AnnexArea' TEXT 'Future Annexation' # # #",


<Result 'TransitOutliers'>

In [74]:
#Populate the County field

arcpy.management.CalculateFields(
    in_table=transit_toAppend,
    expression_type="PYTHON3",
    fields="""County "'Utah'" # #; City '' # #""",
    code_block="",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateFields(
    in_table=transit_toAppend,
    expression_type="PYTHON3",
    fields="""PlanYear "'2014'" # #""",
    code_block="",
    enforce_domains="NO_ENFORCE_DOMAINS"
)


#data = 'https://data.magutah.gov/datasets/a359252b6a364b8fb6e9d66ff0063542_0/explore'

#Data Source
#with arcpy.da.UpdateCursor(transit_toAppend, ["DataSource"]) as cursor:
#    for row in cursor:
#      # Update the field with the hyperlink
#        row[0] = data
#        cursor.updateRow(row)
#print("Hyperlinks have been successfully entered as strings in the attribute table.")

<Result 'TransitOutliers'>

In [75]:
non_city_dict = {
    'Arterial': 'Other Principal Arterial',
    'Collector': 'Minor Collector',
    'Mobility': 'Minor Arterial',
}

fields = ["CityRdType","GenRdType"]  # Ensure these match your dataset

with arcpy.da.UpdateCursor(transit_toAppend, fields) as cursor:
    for row in cursor:
        original_value = row[0]  # Store original for debugging
        if original_value is None or str(original_value).strip() == "":
            row[1] = "Local"  # Default for missing values
        else:
            city_rd_type = str(original_value).strip()  # Normalize spacing
            row[1] = non_city_dict.get(city_rd_type, "Local")  # Default to Local if not found
        
        # Debugging prints
        print(f"Original: {original_value}, Normalized: {city_rd_type}, Updated: {row[1]}")
        
        cursor.updateRow(row)

Original: Arterial, Normalized: Arterial, Updated: Other Principal Arterial
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Collector, Normalized: Collector, Updated: Minor Collector
Original: Arterial, Normali

In [76]:
with arcpy.EnvManager(outputCoordinateSystem=coordinate_system):
    arcpy.management.Append(
        inputs="TransitOutliers",
        target="TransportationMasterPlans_2024",
        schema_type="TEST",
        field_mapping=None,
        subtype="",
        expression="",
        match_fields=None,
        update_geometry="NOT_UPDATE_GEOMETRY",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

print(arcpy.GetMessages())

Start Time: Monday, March 17, 2025 4:27:22 PM
86 row(s) from C:\Users\eclinton\Documents\ArcGIS\Projects\MAG_TransportationUpdate\MAG_TransportationUpdate.gdb\TransitOutliers were appended to C:\Users\eclinton\Documents\ArcGIS\Projects\MAG_TransportationUpdate\MAG_TransportationUpdate.gdb\TransportationMasterPlans_2024
Succeeded at Monday, March 17, 2025 4:27:23 PM (Elapsed Time: 0.65 seconds)


In [77]:
#PRINT UNIQUE VALUES FROM CITY AND GENERALIZED ROAD FIELDS, IF APPLICABLE, AND SEE IF THEY MATCH WITH UDOT (UPDATE FOR EACH TOWN)

## the feature class or table that contains the field (UPDATE/RESET)
input_tbl = transit_toAppend

## the field name to get unique values for (UPDATE/RESET)
field_name = "CityRdType"

unique_values = [
    row[0] for row in arcpy.da.SearchCursor(
                        in_table = input_tbl,
                        field_names = field_name,
                        sql_clause=("DISTINCT", None)
                      )
]

print(unique_values)

['Arterial', 'Collector', 'Mobility']
